In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.utils import *


In [0]:
 #dbutils.library.restartPython()

In [0]:
# construção da tabela histórica do preço de petróleo bruto
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.preco_atual_gasolina"

URL = "https://precos.petrobras.com.br/sele%C3%A7%C3%A3o-de-estados-gasolina"

# scrapping dos dados
PRECO_MEDIO_SEMANAL, DATA_INICIAL, DATA_FINAL = cotacao_gasolina_site_petrobras(URL)



In [0]:
# construção do dataframe com os dados do scrapping
df_cotacao_atual = pd.DataFrame(
    [[DATA_INICIAL, DATA_FINAL,PRECO_MEDIO_SEMANAL]],
columns=['DATA_INICIAL', 'DATA_FINAL','PRECO_MEDIO_SEMANAL'])
df_cotacao_atual

# construção da tabela no formato spark
df_cotacao_atual = criar_tabela_spark(df=df_cotacao_atual)
df_cotacao_atual.show()

In [0]:
# persistência da tabela na camada bronze
df = salvar_tabela_delta_spark(df_cotacao_atual, 
                         mode='append', partitionby=['DATA_FINAL'],merge_schema=True, path=TABELA_INCREMENTAL_RAW)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.precos_gasolina_incremental
LIMIT 10